<a href="https://colab.research.google.com/github/Jibrangit/DL_Assignments/blob/HW6/HW6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten, UpSampling2D, ZeroPadding2D

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import matplotlib.pyplot as plt
from cv2 import resize
import random

def random_rearrange (X_tr, y_tr, seed):
    np.random.seed(seed)
    np.random.shuffle(X_tr)
    np.random.seed(seed)
    np.random.shuffle(y_tr)

def rechannel(x):
  rechanelled_img = np.reshape(np.repeat(x, 3, axis=1), [48, 48, 3])
  return rechanelled_img
  
y_load = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/ages.npy')
x_loaded = np.load('/content/drive/MyDrive/Colab Notebooks/HW6/faces.npy')
dataset_size = len(x_loaded)

x_load = np.zeros([dataset_size, 48, 48, 3])
for i in range(0, dataset_size):
  x_load[i, :, :, :] = rechannel(x_loaded[i, :, :])

no_data = int(x_load.shape[0])
train_size = int(0.8*0.8*no_data)
test_size = int(0.2*no_data)
print(y_load[0])   
random_rearrange(x_load, y_load, 0)
print(y_load[0])                        #Just checking if shuffling ahs been done coorrectly.
(x_test, x_train, x_valid) = x_load[:test_size], x_load[test_size:test_size+train_size],  x_load[test_size+train_size:]
(y_test, y_train, y_valid) = y_load[:test_size], y_load[test_size:test_size+train_size], y_load[test_size+train_size:]
x_train = x_train.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

40.0
48.0


In [4]:
model = Sequential()

In [5]:
from keras.applications import VGG16

model.add(ZeroPadding2D(4, input_shape=(48, 48, 3)))
model.add(UpSampling2D(size=(4, 4)))
model.build()

model.add(VGG16(weights='imagenet', include_top=False, input_shape=[224, 224, 3]))

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=1, activation="relu")) 

58892288/58889256 [==============================] - 0s 0us/step


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 56, 56, 3)         0         
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 4096)              102764544 
_________________________________________________________________
dense_1 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 4

In [8]:
import keras.backend as K
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))

opt = Adam(lr=0.0001)
model.compile(
  loss=root_mean_squared_error,
  optimizer=opt,
  metrics=[RootMeanSquaredError()]
)


In [11]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(x_train,
          y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_valid, y_valid),
          callbacks=[checkpointer])

Epoch 1/5
150/150 [==============================] - 64s 425ms/step - loss: 11.0916 - root_mean_squared_error: 11.2282 - val_loss: 10.6530 - val_root_mean_squared_error: 10.7703

Epoch 00001: val_loss improved from inf to 10.65300, saving model to model.weights.best.hdf5
Epoch 2/5
150/150 [==============================] - 66s 440ms/step - loss: 10.0716 - root_mean_squared_error: 10.1683 - val_loss: 10.3744 - val_root_mean_squared_error: 10.4533

Epoch 00002: val_loss improved from 10.65300 to 10.37441, saving model to model.weights.best.hdf5
Epoch 3/5
150/150 [==============================] - 67s 448ms/step - loss: 9.4701 - root_mean_squared_error: 9.5811 - val_loss: 10.3043 - val_root_mean_squared_error: 10.4153

Epoch 00003: val_loss improved from 10.37441 to 10.30432, saving model to model.weights.best.hdf5
Epoch 4/5
150/150 [==============================] - 68s 454ms/step - loss: 8.6439 - root_mean_squared_error: 8.7444 - val_loss: 10.2187 - val_root_mean_squared_error: 10.3231


In [12]:
# Load the weights with the best validation accuracy
model.load_weights('model.weights.best.hdf5')
# Evaluate the model on test set
score = model.evaluate(x_test, y_test, verbose=0)
# Print test accuracy
print('\n', 'Test loss:', score)
y_hat = model.predict(x_test)
print(y_hat[455:996])
print(y_test[0])


 Test loss: [10.315710067749023, 10.398466110229492]
[[14.623301 ]
 [41.615837 ]
 [42.744785 ]
 [44.92549  ]
 [43.750854 ]
 [32.123867 ]
 [44.043636 ]
 [39.569355 ]
 [37.98228  ]
 [33.189274 ]
 [44.063885 ]
 [62.177654 ]
 [30.147593 ]
 [34.79191  ]
 [39.461964 ]
 [34.96393  ]
 [39.792355 ]
 [44.916    ]
 [20.380817 ]
 [34.81029  ]
 [26.282598 ]
 [42.106647 ]
 [37.524822 ]
 [30.731012 ]
 [30.2082   ]
 [65.74199  ]
 [36.551907 ]
 [59.149742 ]
 [36.348072 ]
 [25.934525 ]
 [32.46836  ]
 [32.65044  ]
 [54.125458 ]
 [40.614677 ]
 [44.2203   ]
 [31.520912 ]
 [51.54232  ]
 [29.572906 ]
 [55.79819  ]
 [ 6.94557  ]
 [36.78813  ]
 [42.200836 ]
 [55.76509  ]
 [32.627094 ]
 [37.936024 ]
 [56.07607  ]
 [49.696    ]
 [33.31516  ]
 [33.90411  ]
 [27.717001 ]
 [36.515877 ]
 [28.446608 ]
 [31.17633  ]
 [34.739082 ]
 [50.43642  ]
 [39.49763  ]
 [18.183039 ]
 [55.591072 ]
 [31.981329 ]
 [28.316078 ]
 [26.589668 ]
 [27.236326 ]
 [37.841045 ]
 [46.791584 ]
 [48.926666 ]
 [51.343857 ]
 [50.373394 ]
 [31.857